In [1]:
import numpy as np
from importnb import Notebook

with Notebook(): 
    import reproduccion_ulutas as rp

[1 2]
1 antibodies were added to the antibodies pool
[0.48571428571428577, 0.641025641025641, 0.4871794871794872, 0.5945945945945945]
Se han borrado 2 anticuerpos
[[ 1  8 11  0  0 12  0  9 10  0  6  2  4  7  3  5]]


In [2]:
POPULATION_SIZE = 3   # indicate the initial size of antibodies population
N = 0.5 # % of antibodies with highest affinities selected to be cloned
R = 0.5 # % of best cloned antibodies to the pool of antibodies
B = 0.5 # worst % of the whole population (RECEPTOR EDITING)
dataset = 'data/instances/testset_a/5x7_Waghodekar_Sahu(1984)[Problem-2].txt'
MAX_ITERATIONS = 1

log = True # Boolean: print logs at the execution of the algorithm.

## 1. Select N% of antibodies with highest affinities. FALTA POR INCLUIR SELECTION ROULETTE WHEEL
        antibodies that have higher efficacy values have a higher selection probability.

## 2. Incluir semilla en la **mutación** para que los resultados sean reproducibles
        Mutation: if new efficacy value is higher, antibody is updated.

        During calculation of the efficacy value, a penalty is defined because Cell#2 does not include any part. 
        Likewise, if a cell does not include any machine, a penalty value is defined and antibody’s objective function value gets a lower efficacy value. 
        The penalty value reduces the chance of the antibody to be selected in the next iteration

Preguntas: 

## 3. Add new random antibodies into the population --> La misma cantidad de anticuerpos que hemos borrado en el apartado anterior 

## 4. Termination criteria
Algorithm is terminated if no improvement is obtained in the efficacy value after 1000 iterations

## 5. Definición de anticuerpos válidos: singleton and residual cells
        each cell must contain at least one part and one machine.

## 6. 

In [7]:
#reads data and translates it into part-machine matrix

matrix, m, p, columns, rows = rp.part_machine_incidence_matrix(dataset)
if log == True: print("\nPart-machine matrix\n",matrix)
#generation of initial pop.
antibodies = rp.generation_initial_population(p = p, m= m, population_size = POPULATION_SIZE, seed = 2021)
if log == True: print("\nAnticuerpos iniciales\n",antibodies)

iterations = 0
while iterations <= MAX_ITERATIONS:
    if log == True: print("\n Iteration number \n",iterations+1)
    #identifies cells in the antibodies
    total_cells = rp.cell_identification(antibodies = antibodies)
    if log == True: print(total_cells)

    #translates numbers into parts and machines
    total_machines, total_parts = rp.decode_cells(total_cells = total_cells) 

    #calculates part-machine matrix for each antibody
    antibody_matrices = rp.create_machine_part_matrix(matrix = matrix, 
                                                    antibodies = antibodies, 
                                                    total_machines = total_machines, 
                                                    total_parts = total_parts)
    #evaluates efficacie of each antibody
    efficacies, voids, exceptions = rp.evaluate_antibodies(antibody_matrices = antibody_matrices, total_cells = total_cells)
    if log == True: print(efficacies)

    # Select N% of antibodies with highest affinities. FALTA POR INCLUIR SELECTION ROULETTE WHEEL
    cloned_antibodies, positions_antibodies_selected = rp.antibodies_selection(antibodies=antibodies, N=N)
    if log == True: print("\nAnticuerpos Clonados\n",cloned_antibodies)

    # Mutation
    rp.mutate_cloned_antibodies(cloned_antibodies = cloned_antibodies)
    if log == True: print("\nAnticuerpos Clonados Mutados\n",cloned_antibodies)

    #CLONES: identifies cells in the antibody
    cloned_total_cells = rp.cell_identification(cloned_antibodies)
    #CLONES: translates numbers into parts and machines
    cloned_total_machines, cloned_total_parts = rp.decode_cells(cloned_total_cells)
    #CLONES: calculates part-machine matrix for each antibody
    cloned_antibody_matrices = rp.create_machine_part_matrix(matrix = matrix,
                                                            antibodies = cloned_antibodies, 
                                                            total_machines = cloned_total_machines, 
                                                            total_parts = cloned_total_parts)
                                                              
    #CLONES evaluates efficacie of each antibody
    cloned_efficacies, cloned_voids, cloned_exceptions = rp.evaluate_antibodies(cloned_antibody_matrices, cloned_total_cells)
    if log == True: print("\nEfficacies",cloned_efficacies)
    # Add R% of best cloned antibodies to the pool of antibodies
    antibodies_pool, updated_efficacies = rp.select_best_cloned_antibodies(antibodies = antibodies,
                                                                        cloned_antibodies = cloned_antibodies,
                                                                        efficacies = efficacies, cloned_efficacies = cloned_efficacies, R = R)
    if log == True: print("\n New antibodies pool \n",antibodies_pool)                                                                        
    # Remove worst members of the antibodies pool (receptor editing)
    antibodies_pool = rp.receptor_editing(antibodies_pool = antibodies_pool, 
                                        updated_efficacies = updated_efficacies, 
                                        B = B)
    if log == True: print("\n Antibodies pool after removal \n", antibodies_pool)
    #generate new random antibodies
    number_new_random_antibodies = round(len(antibodies_pool)*B)
    new_random_antibodies = rp.generation_initial_population(p = p, 
                                                        m = m, 
                                                        population_size = number_new_random_antibodies, 
                                                        seed = 1995)
    if log == True: print("\n New random antibodies \n",new_random_antibodies)                                                        
    antibodies = np.concatenate((antibodies_pool, new_random_antibodies), axis = 0)
    if log == True: print("\n Antibodies pool after adding new random antibodies \n",antibodies)
    #update iteration counter
    iterations = iterations + 1


#select best antibody. FALTA UNA EVALUACIÓN FINAL DE L



Part-machine matrix
     M1  M2  M3  M4  M5
P1   1   0   0   1   0
P2   0   1   0   1   1
P3   0   1   1   1   0
P4   0   1   1   1   1
P5   1   1   1   0   1
P6   1   0   1   0   1
P7   1   0   0   0   0

Anticuerpos iniciales
 [[ 4  0  0  3 11  2  8  0 12  9  0  7  1 10  6  5]
 [ 3  1  5  0  0  0  0 10  7 12  6  8  4  9  2 11]
 [ 0  4 11  5  6  1 10  9  8  7  2  0  3 12  0  0]]

 Iteration number 
 1
[[[4], [3, 11, 2, 8], [12, 9], [7, 1, 10, 6, 5]], [[3, 1, 5], [10, 7, 12, 6, 8, 4, 9, 2, 11]], [[4, 11, 5, 6, 1, 10, 9, 8, 7, 2], [3, 12]]]
[0.4871794871794872, 0.5909090909090909, 0.5957446808510638]

Anticuerpos Clonados
 [[ 0  4 11  5  6  1 10  9  8  7  2  0  3 12  0  0]
 [ 4  0  0  3 11  2  8  0 12  9  0  7  1 10  6  5]]

Anticuerpos Clonados Mutados
 [[ 0  4 11  5  6  1 10  9  7  8  2  0  3 12  0  0]
 [ 0  0  0  3 11  2  8  0 12  9  4  7  1 10  6  5]]

Efficacies [0.5957446808510638, 0.5681818181818182]
1 antibodies were added to the antibodies pool

 New antibodies pool 
 [[ 4  0 

In [4]:
efficacies

[0.5957446808510638, 0.5957446808510638, 0.48571428571428577]